In [1]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### VBF HH production

#### Pythia8 fragments for HH2b2gamma
_PSWeights_ added!

In [2]:
pythia_fragment_run2_CUEP8M1 = """from Configuration.Generator.Pythia8CommonSettings_cfi import *                                                                                                                                             
from Configuration.Generator.Pythia8CUEP8M1Settings_cfi import *                                                                                                                                            
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *                                                                                                                           
                                                                                                                                                                                                            
generator = cms.EDFilter("Pythia8HadronizerFilter",                                                                                                                                                         
                         maxEventsToPrint = cms.untracked.int32(1),                                                                                                                                         
                         pythiaPylistVerbosity = cms.untracked.int32(1),                                                                                                                                    
                         filterEfficiency = cms.untracked.double(1.0),                                                                                                                                      
                         pythiaHepMCVerbosity = cms.untracked.bool(False),                                                                                                                                  
                         comEnergy = cms.double(13000.),                                                                                                                                                    
                         PythiaParameters = cms.PSet(                                                                                                                                                       
        pythia8CommonSettingsBlock,                                                                                                                                                                         
        pythia8CUEP8M1SettingsBlock,                                                                                                                                                                        
        pythia8PSweightsSettingsBlock,                                                                                                                                                                      
        processParameters = cms.vstring(                                                                                                                                                                    
             __CHANNEL_DECAY_FRAGMENT__                                                                                                                                                                     
          ),                                                                                                                                                                                                
        parameterSets = cms.vstring('pythia8CommonSettings',                                                                                                                                                
                                    'pythia8CUEP8M1Settings',                                                                                                                                               
                                    'pythia8PSweightsSettings',                                                                                                                                             
                                    'processParameters'                                                                                                                                                     
                                    )                                                                                                                                                                       
        )                                                                                                                                                                                                   
                         )                                                                                                                                                                                  
                                                                                                                                                                                                            
                                                                                                                                                                                                            
ProductionFilterSequence = cms.Sequence(generator)"""

pythia_fragment_run2_CP5 = """import FWCore.ParameterSet.Config as cms                                                                                                                                      
from Configuration.Generator.Pythia8CommonSettings_cfi import *                                                                                                                                             
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *                                                                                                                                     
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *                                                                                                                           
                                                                                                                                                                                                            
generator = cms.EDFilter("Pythia8HadronizerFilter",                                                                                                                                                         
                         maxEventsToPrint = cms.untracked.int32(1),                                                                                                                                         
                         pythiaPylistVerbosity = cms.untracked.int32(1),                                                                                                                                    
                         filterEfficiency = cms.untracked.double(1.0),                                                                                                                                      
                         pythiaHepMCVerbosity = cms.untracked.bool(False),                                                                                                                                  
                         comEnergy = cms.double(13000.),                                                                                                                                                    
                         PythiaParameters = cms.PSet(                                                                                                                                                       
        pythia8CommonSettingsBlock,                                                                                                                                                                         
        pythia8CP5SettingsBlock,                                                                                                                                                                            
        pythia8PSweightsSettingsBlock,                                                                                                                                                                      
        processParameters = cms.vstring(                                                                                                                                                                    
            __CHANNEL_DECAY_FRAGMENT__                                                                                                                                                                      
          ),                                                                                                                                                                                                
        parameterSets = cms.vstring('pythia8CommonSettings',                                                                                                                                                
                                    'pythia8CP5Settings',                                                                                                                                                   
                                    'pythia8PSweightsSettings',                                                                                                                                             
                                    'processParameters'                                                                                                                                                     
                                    )                                                                                                                                                                       
        )                                                                                                                                                                                                   
                         )                                                                                                                                                                                  
                                                                                                                                                                                                            
ProductionFilterSequence = cms.Sequence(generator)                                                                                                                                                          
"""

In [3]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_run2_CUEP8M1.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_run2_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_run2_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [4]:
LHEproducer = """import FWCore.ParameterSet.Config as cms                                                                                                                                                  
# link to card:                                                                                                                                                                                             
# https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/VBF_HH                                                                                                                                                                                                                                                                                                                                                                                                    
                                                                                                                                                                                                            
externalLHEProducer = cms.EDProducer("ExternalLHEProducer",                                                                                                                                                 
    nEvents = cms.untracked.uint32(5000),                                                                                                                                                                   
    outputFile = cms.string('cmsgrid_final.lhe'),                                                                                                                                                           
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh'),                                                                                                       
    numberOfParameters = cms.uint32(1),                                                                                                                                                                     
    args = cms.vstring('__GRIDPACK__')                                                                                                                                                                      
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
"""


In [5]:
params_list = [['1', '1', '1'], ['1', '1', '0'], ['1', '1', '2'], ['1', '2', '1'], ['0_5', '1', '1'], ['1_5', '1', '1'], ['1', '0', '1']]
years = ["2016", "2017", "2018"]
gridpacks_dict = {}
dataset_names = {}

In [7]:
for year in range(len(years)):
    if (years[year] == "2016"):
        t_datasetname_year = 'VBFHHTo2B2G_CV_{cv}_C2V_{c2v}_C3_{c3}_dipoleRecoilOff-TuneCUETP8M1_PSweights_13TeV-madgraph-pythia8'
        t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/pre2017/13TeV/madgraph/V5_2.6.5/VBF_HH_CV_{cv}_C2V_{c2v}_C3_{c3}_13TeV-madgraph/v1/VBF_HH_CV_{cv}_C2V_{c2v}_C3_{c3}_13TeV-madgraph_slc6_amd64_gcc630_CMSSW_9_3_16_tarball.tar.xz"    
    elif (years[year] == "2017" or years[year] == "2018"):
        t_datasetname_year = 'VBFHHTo2B2G_CV_{cv}_C2V_{c2v}_C3_{c3}_dipoleRecoilOff-TuneCP5_PSweights_13TeV-madgraph-pythia8'
        t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.6.5/VBF_HH_CV_{cv}_C2V_{c2v}_C3_{c3}_13TeV-madgraph/v1/VBF_HH_CV_{cv}_C2V_{c2v}_C3_{c3}_13TeV-madgraph_slc6_amd64_gcc630_CMSSW_9_3_16_tarball.tar.xz"
    
    dataset_names_year = []
    gp_vbf_year = "gp_vbf_" + years[year]
    print(gp_vbf_year)
    gp_vbf_year = []
    

    for i in range(len(params_list)):
        d = {"cv":params_list[i][0], "c2v": params_list[i][1], "c3": params_list[i][2]}
        gp_vbf_year.append(t_gp_year.format_map(d))
        dataset_names_year.append(t_datasetname_year.format_map(d))
        gridpacks_dict.update( {years[year]:gp_vbf_year})
        dataset_names.update( {years[year]:dataset_names_year} )

#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

gp_vbf_2016
gp_vbf_2017
gp_vbf_2018


In [7]:
vbf_hh_2b2gamma = '''                                                                                                                                                                                           
'25:m0 = 125.0',
'25:onMode = off',
'25:onIfMatch = 5 -5',
'25:onIfMatch = 22 22',
'ResonanceDecayFilter:filter = on',
'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
'ResonanceDecayFilter:mothers = 25',   #list of mothers not specified => count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
'ResonanceDecayFilter:daughters = 5,5,22,22'
'''
#'SpaceShower:dipoleRecoil = off' 
#pythia_fragmets_dict = {'VBF_HH' : fragmentsDictCreator(vbf_hh_bbbb)}
#production_modes_events = {'VBF_HH' : 4000000}


## .csv Production

In [8]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {'VBF_HH2b2g' : fragmentsDictCreator(vbf_hh_2b2gamma)}
process = 'VBF_HH2b2g'

In [9]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open(process+'_'+year+'_dipoleRecoilOff.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator'])
        
        for i in range (0,7):
            #print(gridpacks_dict[year][i])
            #print(dataset_names[year][i])
            #version = MadgraphVersion.search(path).group(0)
            version="2.6.5"
            dataset_name = dataset_names[year][i]
            events=4000000
            final_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][i]) + '\n' + process_pythia_map[process][year]
            note = dataset_name.replace('_',' ')
            generators="Madgraph_" + version + "  Pythia8"
            csvwriter.writerow([dataset_name, events, final_fragment, note, generators])